In [1]:
import pandas as pd
import numpy as np
import time
import os

# CVAP

Code mostly from https://github.com/nonpartisan-redistricting-datahub/cvap/blob/main/disagg2022.ipynb by Spencer

## Step 1 Download Data
CVAP comes from the special tabulation from the Census Bereau. The 2011-2015 American Community Survey (ACS) 5-year estimates and this CVAP special tabulation created from those estimates use the same geography source. The reference date for all geographies in this product is January 1, 2015. The best/accurate would be year 2013 for this file. The 2015 file can be downloaded from (https://www.census.gov/programs-surveys/decennial-census/about/voting-rights/cvap.2015.html#list-tab-1518558936)

In [3]:
csv_path = './CVAP_2011-2015_ACS_csv_files/BlockGr.csv'

blockgr_df = pd.read_csv(csv_path, encoding='latin1')

blockgr_df.head()

,GEONAME,lntitle,geoid,lnnumber,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE
0,"Block Group 1, Census Tract 201, Autauga Count...",Total,15000US010010201001,1,625,201,440,113
1,"Block Group 1, Census Tract 201, Autauga Count...",Not Hispanic or Latino,15000US010010201001,2,625,201,440,113
2,"Block Group 1, Census Tract 201, Autauga Count...",American Indian or Alaska Native Alone,15000US010010201001,3,0,11,0,11
3,"Block Group 1, Census Tract 201, Autauga Count...",Asian Alone,15000US010010201001,4,0,11,0,11
4,"Block Group 1, Census Tract 201, Autauga Count...",Black or African American Alone,15000US010010201001,5,90,95,65,68


In [4]:
blockgr_df['lntitle'].unique()

array(['Total', 'Not Hispanic or Latino',
       'American Indian or Alaska Native Alone', 'Asian Alone',
       'Black or African American Alone',
       'Native Hawaiian or Other Pacific Islander Alone', 'White Alone',
       'American Indian or Alaska Native and White', 'Asian and White',
       'Black or African American and White',
       'American Indian or Alaska Native and Black or African American',
       'Remainder of Two or More Race Responses', 'Hispanic or Latino'],
      dtype=object)

In [5]:
blockgr_df['GEONAME'][0]

'Block Group 1, Census Tract 201, Autauga County, Alabama'

## Step 2: Processing the 2021 Block Group CVAP data
CVAP data was retrieved at the block group level.  
The data was pivoted from narrow to wide data based on GEOIDs so that one row is one block group, and each field represents a particular race/ethnicity.  
The fields were renamed to fit character length requirements. 

In [6]:
blockgr_df.head(1)

,GEONAME,lntitle,geoid,lnnumber,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE
0,"Block Group 1, Census Tract 201, Autauga Count...",Total,15000US010010201001,1,625,201,440,113


### Pivot the dataframe, reorder, and combine some fileds

In [7]:
rename_map = {
    "Total": "TOT15",
    "Not Hispanic or Latino": "NHS15",
    "American Indian or Alaska Native Alone": "AIA15",
    "Asian Alone": "ASN15",
    "Black or African American Alone": "BLK15",
    "Native Hawaiian or Other Pacific Islander Alone": "NHP15",
    "White Alone": "WHT15",
    "American Indian or Alaska Native and White": "AIW15",
    "Asian and White": "ASW15",
    "Black or African American and White": "BLW15",
    "American Indian or Alaska Native and Black or African American": "AIB15",
    "Remainder of Two or More Race Responses": "2OM15",
    "Hispanic or Latino": "HSP15",
}


pivot_df = blockgr_df.assign(
    field_name=lambda x: x["lntitle"].map(rename_map)
).pivot(
    index="geoid",
    columns="field_name",
    values=["CVAP_EST", "CIT_EST"]
)

pivot_df.columns = [
    f"{col[0]}_{col[1]}" for col in pivot_df.columns.to_flat_index()
]

pivot_df = pivot_df.reset_index()

field_mapping = {
    "CVAP_EST_TOT15": "CVAP_TOT15",
    "CVAP_EST_NHS15": "CVAP_NHS15",
    "CVAP_EST_AIA15": "CVAP_AIA15",
    "CVAP_EST_ASN15": "CVAP_ASN15",
    "CVAP_EST_BLK15": "CVAP_BLK15",
    "CVAP_EST_NHP15": "CVAP_NHP15",
    "CVAP_EST_WHT15": "CVAP_WHT15",
    "CVAP_EST_AIW15": "CVAP_AIW15",
    "CVAP_EST_ASW15": "CVAP_ASW15",
    "CVAP_EST_BLW15": "CVAP_BLW15",
    "CVAP_EST_AIB15": "CVAP_AIB15",
    "CVAP_EST_2OM15": "CVAP_2OM15",
    "CVAP_EST_HSP15": "CVAP_HSP15",
    "CIT_EST_TOT15": "C_TOT15",
    "CIT_EST_NHS15": "C_NHS15",
    "CIT_EST_AIA15": "C_AIA15",
    "CIT_EST_ASN15": "C_ASN15",
    "CIT_EST_BLK15": "C_BLK15",
    "CIT_EST_NHP15": "C_NHP15",
    "CIT_EST_WHT15": "C_WHT15",
    "CIT_EST_AIW15": "C_AIW15",
    "CIT_EST_ASW15": "C_ASW15",
    "CIT_EST_BLW15": "C_BLW15",
    "CIT_EST_AIB15": "C_AIB15",
    "CIT_EST_2OM15": "C_2OM15",
    "CIT_EST_HSP15": "C_HSP15",
}

pivot_df_renamed = pivot_df.rename(columns=field_mapping)

pivot_df_renamed = pivot_df_renamed.rename(columns={"geoid": "GEOID20"})

# Modify three categories to correspond with the Office of Management and Budget (OMB) racial categories
pivot_df_renamed["CVAP_AIA15"] += pivot_df_renamed["CVAP_AIB15"] + pivot_df_renamed["CVAP_AIW15"]
pivot_df_renamed["CVAP_BLK15"] += pivot_df_renamed["CVAP_AIB15"] + pivot_df_renamed["CVAP_BLW15"]
pivot_df_renamed["CVAP_ASN15"] += pivot_df_renamed["CVAP_ASW15"]

pivot_df_renamed["C_AIA15"] += pivot_df_renamed["C_AIB15"] + pivot_df_renamed["C_AIW15"]
pivot_df_renamed["C_BLK15"] += pivot_df_renamed["C_AIB15"] + pivot_df_renamed["C_BLW15"]
pivot_df_renamed["C_ASN15"] += pivot_df_renamed["C_ASW15"]

# Reorder the columns
desired_order = [
    "GEOID20",
    "CVAP_TOT15", "CVAP_NHS15", "CVAP_AIA15", "CVAP_ASN15", "CVAP_BLK15",
    "CVAP_NHP15", "CVAP_WHT15", "CVAP_AIW15", "CVAP_ASW15", "CVAP_BLW15",
    "CVAP_AIB15", "CVAP_2OM15", "CVAP_HSP15",
    "C_TOT15", "C_NHS15", "C_AIA15", "C_ASN15", "C_BLK15",
    "C_NHP15", "C_WHT15", "C_AIW15", "C_ASW15", "C_BLW15",
    "C_AIB15", "C_2OM15", "C_HSP15"
]

# Reorder columns in the DataFrame
pivot_df_ordered = pivot_df_renamed.reindex(columns=desired_order)

pivot_df_ordered

,GEOID20,CVAP_TOT15,CVAP_NHS15,CVAP_AIA15,CVAP_ASN15,CVAP_BLK15,CVAP_NHP15,CVAP_WHT15,CVAP_AIW15,CVAP_ASW15,...,C_ASN15,C_BLK15,C_NHP15,C_WHT15,C_AIW15,C_ASW15,C_BLW15,C_AIB15,C_2OM15,C_HSP15
0,15000US010010201001,440,440,0,0,65,0,375,0,0,...,0,90,0,535,0,0,0,0,0,0
1,15000US010010201002,1060,1060,29,10,39,0,965,25,10,...,25,64,0,1155,25,15,4,0,15,0
2,15000US010010202001,750,750,0,10,535,0,205,0,10,...,55,615,0,385,0,55,0,0,0,0
3,15000US010010202002,915,895,10,0,469,0,415,10,0,...,15,534,0,485,10,0,4,0,0,15
4,15000US010010203001,1695,1695,60,10,405,10,1165,45,0,...,14,455,10,1520,45,4,0,0,80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220328,15000US721537506011,740,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,980
220329,15000US721537506012,2270,25,0,0,0,0,25,0,0,...,0,0,0,25,0,0,0,0,0,2895
220330,15000US721537506013,930,40,0,0,0,0,40,0,0,...,0,0,0,40,0,0,0,0,0,1330
220331,15000US721537506021,1715,20,0,0,0,0,20,0,0,...,0,0,0,20,0,0,0,0,0,1975


#### Remove "15000US" for GEOID20 as it is country code

In [8]:
pivot_df_ordered["GEOID20"] = pivot_df_ordered["GEOID20"].str.replace("15000US", "", n=1)


In [9]:
pd.set_option('display.max_columns', None)

In [10]:
pivot_df_ordered.head()

,GEOID20,CVAP_TOT15,CVAP_NHS15,CVAP_AIA15,CVAP_ASN15,CVAP_BLK15,CVAP_NHP15,CVAP_WHT15,CVAP_AIW15,CVAP_ASW15,CVAP_BLW15,CVAP_AIB15,CVAP_2OM15,CVAP_HSP15,C_TOT15,C_NHS15,C_AIA15,C_ASN15,C_BLK15,C_NHP15,C_WHT15,C_AIW15,C_ASW15,C_BLW15,C_AIB15,C_2OM15,C_HSP15
0,010010201001,440,440,0,0,65,0,375,0,0,0,0,0,0,625,625,0,0,90,0,535,0,0,0,0,0,0
1,010010201002,1060,1060,29,10,39,0,965,25,10,4,0,15,0,1295,1295,29,25,64,0,1155,25,15,4,0,15,0
2,010010202001,750,750,0,10,535,0,205,0,10,0,0,0,0,1055,1055,0,55,615,0,385,0,55,0,0,0,0
3,010010202002,915,895,10,0,469,0,415,10,0,4,0,0,15,1065,1045,10,15,534,0,485,10,0,4,0,0,15
4,010010203001,1695,1695,60,10,405,10,1165,45,0,0,0,40,0,2145,2145,60,14,455,10,1520,45,4,0,0,80,0


#### Break the country CVAP data file into 50 states

In [11]:
import os
import pandas as pd

state_fips_mapping = {
    '01': 'al', '02': 'ak', '04': 'az', '05': 'ar', '06': 'ca', '08': 'co', '09': 'ct',
    '10': 'de', '12': 'fl', '13': 'ga', '15': 'hi', '16': 'id', '17': 'il', '18': 'in',
    '19': 'ia', '20': 'ks', '21': 'ky', '22': 'la', '23': 'me', '24': 'md', '25': 'ma',
    '26': 'mi', '27': 'mn', '28': 'ms', '29': 'mo', '30': 'mt', '31': 'ne', '32': 'nv',
    '33': 'nh', '34': 'nj', '35': 'nm', '36': 'ny', '37': 'nc', '38': 'nd', '39': 'oh',
    '40': 'ok', '41': 'or', '42': 'pa', '44': 'ri', '45': 'sc', '46': 'sd', '47': 'tn',
    '48': 'tx', '49': 'ut', '50': 'vt', '51': 'va', '53': 'wa', '54': 'wv', '55': 'wi', '56': 'wy'
}

pivot_df_ordered['state_fips'] = pivot_df_ordered['GEOID20'].str[0:2]

output_dir = './bg_cvap_2015'
os.makedirs(output_dir, exist_ok=True)

for fips, abbrev in state_fips_mapping.items():
    state_data = pivot_df_ordered[pivot_df_ordered['state_fips'] == fips]
    if not state_data.empty:
        state_file_path = os.path.join(output_dir, f'{abbrev}_cvap_2015_bg.csv')
        state_data.to_csv(state_file_path, index=False)

output_dir


'./bg_cvap_2015'

## Step 3: Disaggregating the 2015 Block Group CVAP data to 2010 Blocks

In [6]:
# Variable to set the year for the CVAP data
y = "15"

# Define the column names
update_dict = {
'P0040001': 'CVAP_TOT' + y,
 'P0040003': 'CVAP_NHS' + y,
 'P0040007': 'CVAP_AIA' + y,
 'P0040008': 'CVAP_ASN' + y,
 'P0040006': 'CVAP_BLK' + y,
 'P0040009': 'CVAP_NHP' + y,
 'P0040005': 'CVAP_WHT' + y,
 'P0040014': 'CVAP_AIW' + y,
 'P0040015': 'CVAP_ASW' + y,
 'P0040013': 'CVAP_BLW' + y,
 'P0040018': 'CVAP_AIB' + y,
 'P0040011': 'CVAP_2OM' + y,
 'P0040002': 'CVAP_HSP' + y,
 'P0020001': 'C_TOT' + y,
 'P0020003': 'C_NHS' + y,
 'P0020007': 'C_AIA' + y,
 'P0020008': 'C_ASN' + y,
 'P0020006': 'C_BLK' + y,
 'P0020009': 'C_NHP' + y,
 'P0020005': 'C_WHT' + y,
 'P0020014': 'C_AIW' + y,
 'P0020015': 'C_ASW' + y,
 'P0020013': 'C_BLW' + y,
 'P0020018': 'C_AIB' + y,
 'P0020011': 'C_2OM' + y,
 'P0020002': 'C_HSP' + y}

In [7]:
# List of total population columns
tot_cols = ['C_TOT15',
 'CVAP_TOT15']

# List of non-total population columns
non_tot_cols = [
    'CVAP_TOT15', 'CVAP_NHS15', 'CVAP_AIA15', 'CVAP_ASN15', 'CVAP_BLK15', 'CVAP_NHP15',
    'CVAP_WHT15', 'CVAP_AIW15', 'CVAP_ASW15', 'CVAP_BLW15', 'CVAP_AIB15', 'CVAP_2OM15',
    'CVAP_HSP15', 'C_TOT15', 'C_NHS15', 'C_AIA15', 'C_ASN15', 'C_BLK15', 'C_NHP15',
    'C_WHT15', 'C_AIW15', 'C_ASW15', 'C_BLW15', 'C_AIB15', 'C_2OM15', 'C_HSP15'
]


In [ ]:
def get_state_bg_cvap(state):
    '''
    Add code to retrieve the bg data
    ''' 
    df = pd.read_csv(os.path.join('./bg_cvap_2015', state+'_cvap_2015_bg.csv'))
    return df
    
# def get_state_block_pl(state):
#     '''
#     Add code to retrieve block-level PL data here
#     '''
#     df = pd.read_csv(os.path.join('./block_pl', state+'_pl2010_b.csv'))
#     df['GEOID20'] = df['GEOID20'].apply(lambda x: str(x).zfill(15))
#     return df

# Create a GEOID20 column for each pl for merge and return the new df
def get_state_block_pl(state):
    '''
    Retrieves block-level PL data for a given state and generates the GEOID20 column.
    '''
    import os
    import pandas as pd
    
    # Read the block-level PL data
    file_path = os.path.join('./block_pl', f"{state}_pl2010_b.csv")
    df = pd.read_csv(file_path)
    
    # Convert all column names to uppercase for consistency
    df.columns = [col.upper() for col in df.columns]
    
    # Verify necessary columns exist
    required_columns = ["STATE", "COUNTY", "TRACT", "BLOCK"]
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Required column '{col}' not found in the file {file_path}. Available columns: {list(df.columns)}")
    
    # Create the GEOID20 column
    df['GEOID20'] = (
        df["STATE"].astype(str).str.zfill(2) + 
        df["COUNTY"].astype(str).str.zfill(3) +
        df["TRACT"].astype(str).str.zfill(6) +
        df["BLOCK"].astype(str).str.zfill(4)
    )
    
    # Verify GEOID20 creation
    print("Sample GEOID20 values:")
    print(df[['STATE', 'COUNTY', 'TRACT', 'BLOCK', 'GEOID20']].head())
    
    return df



In [18]:
pl_hi = get_state_block_pl('hi')
pl_hi.head()

Sample GEOID20 values:
   STATE  COUNTY  TRACT  BLOCK          GEOID20
0     15       1  21800   1000  150010218001000
1     15       1  21800   1001  150010218001001
2     15       1  21800   1002  150010218001002
3     15       1  21800   1004  150010218001004
4     15       1  21800   1005  150010218001005


/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_7193/2401535465.py:24: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,FILEID,STUSAB,SUMLEV,GEOCOMP,CHARITER,LOGRECNO,REGION,DIVISION,STATE,COUNTY,COUNTYCC,COUSUB,COUSUBCC,PLACE,PLACECC,TRACT,BLKGRP,BLOCK,CONCIT,CONCITCC,AIANHH,AIANHHFP,AIANHHCC,AIHHTLI,AITSCE,AITS,AITSCC,TTRACT,TBLKGRP,ANRC,ANRCCC,CBSA,METDIV,C,NECTA,NECTADIV,CNECTA,CBSAPCI,NECTAPCI,CD,SLDU,SLDL,V,VTDI,SDELM,SDSEC,SDUNI,AREALAND,AREAWATR,NAME,FUNCSTAT,POP100,HU100,INTPTLAT,INTPTLON,LSADC,U,STATENS,COUNTYNS,COUSUBNS,PLACENS,CONCITNS,AIANHHNS,AITSNS,ANRCNS,MEMI,NMEMI,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,P0010007,P0010008,P0010009,P0010010,P0010011,P0010012,P0010013,P0010014,P0010015,P0010016,P0010017,P0010018,P0010019,P0010020,P0010021,P0010022,P0010023,P0010024,P0010025,P0010026,P0010027,P0010028,P0010029,P0010030,P0010031,P0010032,P0010033,P0010034,P0010035,P0010036,P0010037,P0010038,P0010039,P0010040,P0010041,P0010042,P0010043,P0010044,P0010045,P0010046,P0010047,P0010048,P0010049,P0010050,P0010051,P0010052,P0010053,P0010054,P0010055,P0010056,P0010057,P0010058,P0010059,P0010060,P0010061,P0010062,P0010063,P0010064,P0010065,P0010066,P0010067,P0010068,P0010069,P0010070,P0010071,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,P0020031,P0020032,P0020033,P0020034,P0020035,P0020036,P0020037,P0020038,P0020039,P0020040,P0020041,P0020042,P0020043,P0020044,P0020045,P0020046,P0020047,P0020048,P0020049,P0020050,P0020051,P0020052,P0020053,P0020054,P0020055,P0020056,P0020057,P0020058,P0020059,P0020060,P0020061,P0020062,P0020063,P0020064,P0020065,P0020066,P0020067,P0020068,P0020069,P0020070,P0020071,P0020072,P0020073,P0030001,P0030002,P0030003,P0030004,P0030005,P0030006,P0030007,P0030008,P0030009,P0030010,P0030011,P0030012,P0030013,P0030014,P0030015,P0030016,P0030017,P0030018,P0030019,P0030020,P0030021,P0030022,P0030023,P0030024,P0030025,P0030026,P0030027,P0030028,P0030029,P0030030,P0030031,P0030032,P0030033,P0030034,P0030035,P0030036,P0030037,P0030038,P0030039,P0030040,P0030041,P0030042,P0030043,P0030044,P0030045,P0030046,P0030047,P0030048,P0030049,P0030050,P0030051,P0030052,P0030053,P0030054,P0030055,P0030056,P0030057,P0030058,P0030059,P0030060,P0030061,P0030062,P0030063,P0030064,P0030065,P0030066,P0030067,P0030068,P0030069,P0030070,P0030071,P0040001,P0040002,P0040003,P0040004,P0040005,P0040006,P0040007,P0040008,P0040009,P0040010,P0040011,P0040012,P0040013,P0040014,P0040015,P0040016,P0040017,P0040018,P0040019,P0040020,P0040021,P0040022,P0040023,P0040024,P0040025,P0040026,P0040027,P0040028,P0040029,P0040030,P0040031,P0040032,P0040033,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073,H0010001,H0010002,H0010003,GEOID20
0,PLST,HI,750,0,0,4498,4,9,15,1,H1,92520,Z5,13600,U1,21800,1,1000,99999,99,9999,99999,99,9,999,99999,99,999999,9,99999,99,25900,99999,999,99999,99999,999,N,N,2,003,001,101,P,99999,99999,30,40545,0,Block 1000,S,6,5,20.236725,-155.832155,BK,99999,1779782,365280,1935665,2414039,99999999,99999999,99999999,99999999,2,9,6,4,3,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,6,4,3,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,4,3,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,6,4,3,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,4,1,150010

In [20]:
def data_rounding(x):
    x = x.sort_values(ascending = False, kind='mergesort')
    floor = np.floor(x)
    diff = x - floor
    to_add = int(round(sum(x),0) - sum(floor))
    floor[diff.nlargest(to_add).index] += 1
    return floor.sort_index(ascending = True).astype(int)

### This is the unmodified funtion with rounding 

In [21]:
def run_disagg_changed_original(state):
    '''Runs a disaggregation to the block-level from CVAP data'''
    
    # Set the state abbreviation to lower case
    state = state.lower()
    
    # Get the block-group CVAP data and block-level PL data
    state_bg_cvap = get_state_bg_cvap(state)
    state_block_pl = get_state_block_pl(state)
    
    # PL-Data: Rename the columns to their proxies using the above
    state_block_pl.rename(columns = update_dict, inplace = True)


    state_block_pl["BLKGRP"] = state_block_pl["GEOID20"].astype(str).str.zfill(15).str[0:12]
    print('state_block_pl.head():')
    print(state_block_pl.head())
    # PL-Data: Create a count variable for number of blocks in block group
    state_block_pl["COUNT"] = 1

    # PL-Data: Aggregate the block groups together, clean indices
    print('len(list(state_block_pl.columns))')
    print(len(list(state_block_pl.columns)))

    for i in state_block_pl.columns:
        if state_block_pl[i].dtype == 'object':
            if i not in ['BLKGRP','GEOID20']:
                state_block_pl.drop(columns = i, inplace=True)

    print('len(list(state_block_pl.columns))')            
    print(len(list(state_block_pl.columns)))

    state_bg_pl = state_block_pl.groupby(["BLKGRP"]).sum()
    state_bg_pl.reset_index(inplace = True, drop = False)
    
    # CVAP-Data: Clean the GEOID20, call it BLKGRP
    state_bg_cvap["GEOID20"] = state_bg_cvap["GEOID20"].astype(str).str.zfill(12)
    state_bg_cvap.rename(columns = {"GEOID20":"BLKGRP"}, inplace = True)

    # Merge the two files together
    merged_data = pd.merge(state_block_pl, state_bg_pl, on = "BLKGRP", how = "left", indicator = "ind_1", suffixes = ["_block","_bg"] )
    merged_data_final = pd.merge(merged_data, state_bg_cvap, on = "BLKGRP", how = "left", indicator = "ind_2")
    merged_data_final = merged_data_final[merged_data_final['ind_2']=='both']
    
#    display(state_bg_cvap[state_bg_cvap['BLKGRP']=='091703431011'])
    # Mapping for total columns
    col_mapping = {"C_TOT15":"P0010001", 'CVAP_TOT15':"P0030001"}
    
    # Iterate over the total columns first
    for val in tot_cols:
        merged_data_final[val+"_DISAGG"] = np.where(merged_data_final[val]==0,0,
                                                    np.where(merged_data_final[val+"_bg"]!=0, (merged_data_final[val+"_block"]/merged_data_final[val+"_bg"]) * merged_data_final[val], 
                                                        np.where(merged_data_final[col_mapping[val]+"_bg"]==0, (1/merged_data_final["COUNT_bg"])*merged_data_final[val],(merged_data_final[col_mapping[val]+"_block"]/merged_data_final[col_mapping[val]+"_bg"]) * merged_data_final[val]) 
                                                            )
                                                    )
                                                                 
    # Iterate over the remaining columns after
    for val in non_tot_cols:
        merged_data_final[val+"_DISAGG"] = np.where(merged_data_final[val]==0,0,
                                                    np.where(merged_data_final[val+"_bg"]!=0, (merged_data_final[val+"_block"]/merged_data_final[val+"_bg"]) * merged_data_final[val], 
                                                        np.where("VAP" in val, (merged_data_final["CVAP_TOT15_DISAGG"]/merged_data_final["CVAP_TOT15"]) * merged_data_final[val],
                                                                    (merged_data_final["C_TOT15_DISAGG"]/merged_data_final["C_TOT15"]) * merged_data_final[val]
                                                                )
                                                            )
                                                        )
    disagg_cols=[i for i in list(merged_data_final.columns) if "_DISAGG" in i]
    merged_data_final.sort_values("GEOID20_block", ascending = False, inplace = True)

    for col in disagg_cols:
        try:
            assert(len(merged_data_final[merged_data_final[col].isna()][['GEOID20_block','BLKGRP']])==0)
        except:
            display(merged_data_final[merged_data_final[col].isna()][['GEOID20_block','BLKGRP']])
        merged_data_final[col+"_rounded"] = merged_data_final.groupby('BLKGRP')[[col]].transform(data_rounding)
        if (~(merged_data_final.groupby("BLKGRP")[col+"_rounded"].transform(sum) == merged_data_final[col.replace("_DISAGG","")])).any():
            print("ISSUE")
    print('rounding complete')        
    merged_data_final_export = merged_data_final[["GEOID20_block"]+[i for i in merged_data_final.columns if "_rounded" in i]]

    print('After rounding')
    display(merged_data_final_export.head())
    merged_data_final_export = merged_data_final
    for i in merged_data_final_export.columns:
        if '_rounded' in i:
            print(i)
            new_name = i.replace('_DISAGG_rounded','')
            print(new_name)
            merged_data_final_export[new_name] = merged_data_final_export[i]
            merged_data_final_export[new_name] = merged_data_final_export[new_name].fillna(0)
            merged_data_final_export[new_name] = merged_data_final_export[new_name].apply(lambda x: int(x))

    print('After making to integers')
    display(merged_data_final_export.head())
    merged_data_final_export = merged_data_final

    # Prepare the export DataFrame
    merged_data_final_export = merged_data_final[["GEOID20_block"] + disagg_cols]

    # Ensure the disaggregated columns are integers
    for col in disagg_cols:
        merged_data_final_export[col] = merged_data_final_export[col].fillna(0).apply(lambda x: int(x) if not pd.isna(x) else 0)

    print('After processing DISAGG columns')
    display(merged_data_final_export.head())
    merged_data_final_export['GEOID20'] = merged_data_final_export['GEOID20_block'].fillna(merged_data_final_export['GEOID20_block'])
    merged_data_final_export['BLKGRP15'] = merged_data_final_export['GEOID20_block'].astype(str).str.zfill(15).str[0:12]

    column_order = ['GEOID20','BLKGRP15','C_TOT15','C_NHS15','C_AIA15','C_ASN15','C_BLK15','C_NHP15','C_WHT15','C_AIW15','C_ASW15','C_BLW15','C_AIB15','C_2OM15','C_HSP15',
                'CVAP_TOT15','CVAP_NHS15','CVAP_AIA15','CVAP_ASN15','CVAP_BLK15','CVAP_NHP15','CVAP_WHT15','CVAP_AIW15','CVAP_ASW15','CVAP_BLW15','CVAP_AIB15','CVAP_2OM15','CVAP_HSP15',]

    os.makedirs(f"./2015_cvap_disagg/{state}/", exist_ok=True)

    merged_data_final_export = merged_data_final_export[column_order]
    print('Exported DF')
    merged_data_final_export.sort_values(by = 'GEOID20',inplace=True,ascending=True)
    display(merged_data_final_export.head())
    # Export to CSV
    merged_data_final_export.to_csv("./2015_cvap_disagg/"+state+"/"+state+"_2015_cvap_block_original.csv", index = False)    



#### This is the modified function with rounding and crosswalk removed

In [5]:
def run_disagg_changed_no_rounding(state):
    '''Runs a disaggregation to the block-level from CVAP data'''
    
    # Set the state abbreviation to lower case
    state = state.lower()
    
    # Get the block-group CVAP data and block-level PL data
    state_bg_cvap = get_state_bg_cvap(state)
    state_block_pl = get_state_block_pl(state)
    
    # PL-Data: Rename the columns to their proxies using the above
    state_block_pl.rename(columns = update_dict, inplace = True)


    state_block_pl["BLKGRP"] = state_block_pl["GEOID20"].astype(str).str.zfill(15).str[0:12]

    # PL-Data: Create a count variable for number of blocks in block group
    state_block_pl["COUNT"] = 1

    # PL-Data: Aggregate the block groups together, clean indices


    for i in state_block_pl.columns:
        if state_block_pl[i].dtype == 'object':
            if i not in ['BLKGRP','GEOID20']:
                state_block_pl.drop(columns = i, inplace=True)


    state_bg_pl = state_block_pl.groupby(["BLKGRP"]).sum()
    state_bg_pl.reset_index(inplace = True, drop = False)
    
    # CVAP-Data: Clean the GEOID20, call it BLKGRP
    state_bg_cvap["GEOID20"] = state_bg_cvap["GEOID20"].astype(str).str.zfill(12)
    state_bg_cvap.rename(columns = {"GEOID20":"BLKGRP"}, inplace = True)

    # Merge the two files together
    merged_data = pd.merge(state_block_pl, state_bg_pl, on = "BLKGRP", how = "left", indicator = "ind_1", suffixes = ["_block","_bg"] )
    merged_data_final = pd.merge(merged_data, state_bg_cvap, on = "BLKGRP", how = "left", indicator = "ind_2")
    merged_data_final = merged_data_final[merged_data_final['ind_2']=='both']
    
    # Mapping for total columns
    col_mapping = {"C_TOT15":"P0010001", 'CVAP_TOT15':"P0030001"}
    
    # Iterate over the total columns first
    for val in tot_cols:
        merged_data_final[val+"_DISAGG"] = np.where(merged_data_final[val]==0,0,
                                                    np.where(merged_data_final[val+"_bg"]!=0, (merged_data_final[val+"_block"]/merged_data_final[val+"_bg"]) * merged_data_final[val], 
                                                        np.where(merged_data_final[col_mapping[val]+"_bg"]==0, (1/merged_data_final["COUNT_bg"])*merged_data_final[val],(merged_data_final[col_mapping[val]+"_block"]/merged_data_final[col_mapping[val]+"_bg"]) * merged_data_final[val]) 
                                                            )
                                                    )
                                                                 
    # Iterate over the remaining columns after
    for val in non_tot_cols:
        merged_data_final[val+"_DISAGG"] = np.where(merged_data_final[val]==0,0,
                                                    np.where(merged_data_final[val+"_bg"]!=0, (merged_data_final[val+"_block"]/merged_data_final[val+"_bg"]) * merged_data_final[val], 
                                                        np.where("VAP" in val, (merged_data_final["CVAP_TOT15_DISAGG"]/merged_data_final["CVAP_TOT15"]) * merged_data_final[val],
                                                                    (merged_data_final["C_TOT15_DISAGG"]/merged_data_final["C_TOT15"]) * merged_data_final[val]
                                                                )
                                                            )
                                                        )
    disagg_cols=[i for i in list(merged_data_final.columns) if "_DISAGG" in i]
    merged_data_final.sort_values("GEOID20_block", ascending = False, inplace = True)

    # Prepare the export DataFrame
    merged_data_final_export = merged_data_final[["GEOID20_block"] + disagg_cols]

    merged_data_final_export['GEOID20'] = merged_data_final_export['GEOID20_block'].fillna(merged_data_final_export['GEOID20_block'])
    merged_data_final_export['BLKGRP15'] = merged_data_final_export['GEOID20_block'].astype(str).str.zfill(15).str[0:12]
    
    merged_data_final_export.rename(columns=lambda x: x.replace('_DISAGG', ''), inplace=True)

    column_order = ['GEOID20','BLKGRP15','C_TOT15','C_NHS15','C_AIA15','C_ASN15','C_BLK15','C_NHP15','C_WHT15','C_AIW15','C_ASW15','C_BLW15','C_AIB15','C_2OM15','C_HSP15',
                'CVAP_TOT15','CVAP_NHS15','CVAP_AIA15','CVAP_ASN15','CVAP_BLK15','CVAP_NHP15','CVAP_WHT15','CVAP_AIW15','CVAP_ASW15','CVAP_BLW15','CVAP_AIB15','CVAP_2OM15','CVAP_HSP15']

    os.makedirs(f"./2015_cvap_disagg/{state}/", exist_ok=True)

    merged_data_final_export = merged_data_final_export[column_order]
    print('Exported DF')
    merged_data_final_export.sort_values(by = 'GEOID20',inplace=True,ascending=True)
    display(merged_data_final_export.head())
    # Export to CSV
    merged_data_final_export.to_csv("./2015_cvap_disagg/"+state+"/"+state+"_2015_cvap_block_no_rounding.csv", index = False)    



In [9]:
# Iterate over the abbreviations
for abbrev in [
    'al', 'ak', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'fl', 'ga', 'hi', 'id', 'il', 'in', 'ia', 
    'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 
    'nm', 'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'vt', 
    'va', 'wa', 'wv', 'wi', 'wy'
]:
    # Start timer
    start_process_time = time.time()
    
    # Run disagg
    run_disagg_changed_no_rounding(abbrev)
    # run_disagg_changed(abbrev)
    
    # End timer
    end_process_time = time.time()
    
    # Write to a .txt file
    print(abbrev + " took " +  str(round(end_process_time - start_process_time,3)) + " seconds ")
    with open('log_no_rounding.txt', 'a') as t:
        t.write(abbrev + " took " +  str(round(end_process_time - start_process_time,3)) + " seconds " + "\n")
    t.close()

/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_1372/2401535465.py:24: DtypeWarning: Columns (22,23,27,28,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Sample GEOID20 values:
   STATE  COUNTY  TRACT  BLOCK          GEOID20
0      1       1  21100   2000  010010211002000
1      1       1  21100   2001  010010211002001
2      1       1  21100   2003  010010211002003
3      1       1  21100   2004  010010211002004
4      1       1  21100   2005  010010211002005


/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_1372/2687523634.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_final_export['GEOID20'] = merged_data_final_export['GEOID20_block'].fillna(merged_data_final_export['GEOID20_block'])
/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_1372/2687523634.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_final_export['BLKGRP15'] = merged_data_final_export['GEOID20_block'].astype(str).str.zfill(15).str[0:12]
/var/fold

Exported DF


,GEOID20,BLKGRP15,C_TOT15,C_NHS15,C_AIA15,C_ASN15,C_BLK15,C_NHP15,C_WHT15,C_AIW15,...,CVAP_ASN15,CVAP_BLK15,CVAP_NHP15,CVAP_WHT15,CVAP_AIW15,CVAP_ASW15,CVAP_BLW15,CVAP_AIB15,CVAP_2OM15,CVAP_HSP15
1754,010010201001000,010010201001,54.620344,56.066176,0.0,0.0,4.615385,0.0,50.385274,0.0,...,0.0,4.727273,0.0,35.714286,0.0,0.0,0.0,0.0,0.0,0.0
1755,010010201001001,010010201001,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1756,010010201001002,010010201001,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1358,010010201001003,010010201001,67.156160,65.257353,0.0,0.0,4.615385,0.0,60.462329,0.0,...,0.0,2.363636,0.0,41.666667,0.0,0.0,0.0,0.0,0.0,0.0
1359,010010201001004,010010201001,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


...

/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_1372/2401535465.py:24: DtypeWarning: Columns (22,23,27,28,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Sample GEOID20 values:
   STATE  COUNTY   TRACT  BLOCK          GEOID20
0     56       1  963900   2202  560019639002202
1     56       1  963900   2203  560019639002203
2     56       1  963900   2204  560019639002204
3     56       1  963900   2208  560019639002208
4     56       1  963900   2209  560019639002209
Exported DF


/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_1372/2687523634.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_final_export['GEOID20'] = merged_data_final_export['GEOID20_block'].fillna(merged_data_final_export['GEOID20_block'])
/var/folders/nq/qm5_bhw92xbcc_69zxcvm2kc0000gn/T/ipykernel_1372/2687523634.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_final_export['BLKGRP15'] = merged_data_final_export['GEOID20_block'].astype(str).str.zfill(15).str[0:12]
/var/fold

,GEOID20,BLKGRP15,C_TOT15,C_NHS15,C_AIA15,C_ASN15,C_BLK15,C_NHP15,C_WHT15,C_AIW15,...,CVAP_ASN15,CVAP_BLK15,CVAP_NHP15,CVAP_WHT15,CVAP_AIW15,CVAP_ASW15,CVAP_BLW15,CVAP_AIB15,CVAP_2OM15,CVAP_HSP15
979,560019627001000,560019627001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
980,560019627001001,560019627001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
981,560019627001002,560019627001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
982,560019627001003,560019627001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
983,560019627001004,560019627001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


wy took 3.248 seconds 


## Step 4: Process and use the 2010_to_2020 crosswalk

#### Process the crosswalk file so that each row is in ['block_fips_2020', 'block_fips_2010', 'weight'] format

In [ ]:
import os
import pandas as pd

def preprocess_crosswalk_grouped(file_path):

    normalized_data = []

    with open(file_path, 'r') as f:
        for line in f:
            # Split the line into fields
            fields = line.strip().split(',')

            # The first field is the 2020 block
            block_2020 = fields[0]

            # Remaining fields come in pairs of [2010_block, weight]
            for i in range(1, len(fields), 2):
                try:
                    block_2010 = fields[i]
                    weight = float(fields[i + 1])
                    normalized_data.append((block_2020, block_2010, weight))
                except IndexError:
                    # Skip incomplete rows
                    print(f"Skipped incomplete row: {fields}")
                    continue

    df = pd.DataFrame(normalized_data, columns=['block_fips_2020', 'block_fips_2010', 'weight'])
    return df

def process_all_crosswalks_grouped(directory, output_directory):

    os.makedirs(output_directory, exist_ok=True)

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            print(f"Processing file: {file_path}")

            # Preprocess the file
            try:
                crosswalk_df = preprocess_crosswalk_grouped(file_path)

                # Save the normalized data
                output_file = os.path.join(output_directory, filename)
                crosswalk_df.to_csv(output_file, index=False)
                print(f"Processed file saved to: {output_file}")
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

input_directory = "./block10block20_crosswalks"
output_directory = "./processed_crosswalks"

process_all_crosswalks_grouped(input_directory, output_directory)


Processing file: ./block10block20_crosswalks/block1020_crosswalk_08.csv
Processed file saved to: ./processed_crosswalks/block1020_crosswalk_08.csv
Processing file: ./block10block20_crosswalks/block1020_crosswalk_34.csv
Processed file saved to: ./processed_crosswalks/block1020_crosswalk_34.csv
Processing file: ./block10block20_crosswalks/block1020_crosswalk_20.csv
Processed file saved to: ./processed_crosswalks/block1020_crosswalk_20.csv
Processing file: ./block10block20_crosswalks/block1020_crosswalk_21.csv
Processed file saved to: ./processed_crosswalks/block1020_crosswalk_21.csv
Processing file: ./block10block20_crosswalks/block1020_crosswalk_35.csv
Processed file saved to: ./processed_crosswalks/block1020_crosswalk_35.csv
Processing file: ./block10block20_crosswalks/block1020_crosswalk_09.csv
Processed file saved to: ./processed_crosswalks/block1020_crosswalk_09.csv
Processing file: ./block10block20_crosswalks/block1020_crosswalk_23.csv
Processed file saved to: ./processed_crosswalk

#### Was planning to use: for each GEOID20, find its corresponding GEOID10 row in the CVAP data, apply the weights and sum them. But some states have more than 100,000 GEOID20, looking up and storing takes too much time.

In [ ]:
def translate_2010_to_2020(cvap_data, crosswalk):

    crosswalk['block_fips_2010'] = crosswalk['block_fips_2010'].apply(lambda x: str(x).zfill(15))
    crosswalk['block_fips_2020'] = crosswalk['block_fips_2020'].apply(lambda x: str(x).zfill(15))
    cvap_data['GEOID10'] = cvap_data['GEOID20'].apply(lambda x: str(x).zfill(15))

    data_columns = [
        col for col in cvap_data.columns
        if col not in ['GEOID20', 'GEOID10', 'BLKGRP15']
    ]

    # Initialize a results dictionary
    results = {col: [] for col in ['GEOID20'] + data_columns}

    # Create a dictionary for quick lookups of CVAP data by GEOID10
    cvap_dict = cvap_data.set_index('GEOID10').to_dict(orient='index')

    # Iterate through unique GEOID20 values in the crosswalk
    for geoid_2020 in crosswalk['block_fips_2020'].unique():
        relevant_rows = crosswalk[crosswalk['block_fips_2020'] == geoid_2020]

        aggregated_row = {col: 0 for col in data_columns}
        aggregated_row['GEOID20'] = geoid_2020

        for _, row in relevant_rows.iterrows():
            block_2010 = row['block_fips_2010']
            weight = row['weight']

            if block_2010 in cvap_dict:
                cvap_row = cvap_dict[block_2010]
            else:
                cvap_row = {col: 0 for col in data_columns}

            for col in data_columns:
                aggregated_row[col] += cvap_row[col] * weight

        for col in ['GEOID20'] + data_columns:
            results[col].append(aggregated_row[col])

    translated_data = pd.DataFrame(results)

    translated_data[data_columns] = translated_data[data_columns].round(2)

    return translated_data

Processing state: AL


KeyboardInterrupt: 

#### Alternatively, use merge:   
 
The merge operation aligns the block_fips_2010 column in the crosswalk with the GEOID10 column in the CVAP dataset.  
The weight column from the crosswalk is retained during the merge operation.  
After merging, the data is grouped by block_fips_2020, which represents the 2020 Census block IDs.  
Aggregation is performed on the redistributed CVAP values within each 2020 block group.  


In [ ]:
def translate_2010_to_2020_optimized(cvap_data, crosswalk):
    # Ensure block identifiers are strings of 15 digits
    crosswalk['block_fips_2010'] = crosswalk['block_fips_2010'].astype(str).str.zfill(15)
    crosswalk['block_fips_2020'] = crosswalk['block_fips_2020'].astype(str).str.zfill(15)
    cvap_data['GEOID10'] = cvap_data['GEOID20'].astype(str).str.zfill(15)

    # Columns to process (exclude GEOID20, GEOID10, BLKGRP15)
    data_columns = [
        col for col in cvap_data.columns if col not in ['GEOID20', 'GEOID10', 'BLKGRP15']
    ]

    # Merge crosswalk with CVAP data
    merged = crosswalk.merge(
        cvap_data,
        left_on='block_fips_2010',
        right_on='GEOID10',
        how='left'
    )

    # Fill missing values in merged CVAP columns with 0
    merged[data_columns] = merged[data_columns].fillna(0)

    # Apply weights
    for col in data_columns:
        merged[col] = merged[col] * merged['weight']

    # Group by block_fips_2020 and sum the weighted values
    aggregated = merged.groupby('block_fips_2020', as_index=False)[data_columns].sum()

    # Rename block_fips_2020 to GEOID20 for output
    aggregated.rename(columns={'block_fips_2020': 'GEOID20'}, inplace=True)

    # Round numeric columns to the nearest hundredth
    aggregated[data_columns] = aggregated[data_columns].round(2)

    return aggregated

def process_all_states_fixed(cvap_dir, crosswalk_dir, output_dir, state_fips_mapping):

    for fips_code, state_abbrev in state_fips_mapping.items():
        print(f"Processing state: {state_abbrev.upper()}")

        cvap_file = os.path.join(cvap_dir, state_abbrev, f"{state_abbrev}_2015_cvap_block_no_rounding.csv")
        crosswalk_file = os.path.join(crosswalk_dir, f"block1020_crosswalk_{fips_code}.csv")
        output_file = os.path.join(output_dir, state_abbrev, f"{state_abbrev}_2020_cvap_block.csv")

        if not os.path.exists(cvap_file):
            print(f"CVAP data file missing for {state_abbrev.upper()}: {cvap_file}")
            continue
        if not os.path.exists(crosswalk_file):
            print(f"Crosswalk file missing for {state_abbrev.upper()}: {crosswalk_file}")
            continue

        cvap_data = pd.read_csv(cvap_file)
        crosswalk = pd.read_csv(crosswalk_file)

        try:
            translated_data = translate_2010_to_2020_optimized(cvap_data, crosswalk)

            translated_data = translated_data[[col for col in translated_data.columns if col != 'BLKGRP15']]

            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            translated_data.to_csv(output_file, index=False)
            print(f"Translated CVAP data saved to {output_file}")

        except Exception as e:
            print(f"Error processing {state_abbrev.upper()}: {e}")

state_fips_mapping = {
    '01': 'al', '02': 'ak', '04': 'az', '05': 'ar', '06': 'ca', '08': 'co', '09': 'ct',
    '10': 'de', '12': 'fl', '13': 'ga', '15': 'hi', '16': 'id', '17': 'il', '18': 'in',
    '19': 'ia', '20': 'ks', '21': 'ky', '22': 'la', '23': 'me', '24': 'md', '25': 'ma',
    '26': 'mi', '27': 'mn', '28': 'ms', '29': 'mo', '30': 'mt', '31': 'ne', '32': 'nv',
    '33': 'nh', '34': 'nj', '35': 'nm', '36': 'ny', '37': 'nc', '38': 'nd', '39': 'oh',
    '40': 'ok', '41': 'or', '42': 'pa', '44': 'ri', '45': 'sc', '46': 'sd', '47': 'tn',
    '48': 'tx', '49': 'ut', '50': 'vt', '51': 'va', '53': 'wa', '54': 'wv', '55': 'wi', '56': 'wy'
}

cvap_dir = "./2015_cvap_disagg"
crosswalk_dir = "./processed_crosswalks"
output_dir = "./2015_cvap_diagg_2020"

# Process all states
process_all_states_fixed(cvap_dir, crosswalk_dir, output_dir, state_fips_mapping)

Processing state: AL
Translated CVAP data saved to ./2015_cvap_diagg_2020/al/al_2020_cvap_block.csv
Processing state: AK
Translated CVAP data saved to ./2015_cvap_diagg_2020/ak/ak_2020_cvap_block.csv
Processing state: AZ
Translated CVAP data saved to ./2015_cvap_diagg_2020/az/az_2020_cvap_block.csv
Processing state: AR
Translated CVAP data saved to ./2015_cvap_diagg_2020/ar/ar_2020_cvap_block.csv
Processing state: CA
Translated CVAP data saved to ./2015_cvap_diagg_2020/ca/ca_2020_cvap_block.csv
Processing state: CO
Translated CVAP data saved to ./2015_cvap_diagg_2020/co/co_2020_cvap_block.csv
Processing state: CT
Translated CVAP data saved to ./2015_cvap_diagg_2020/ct/ct_2020_cvap_block.csv
Processing state: DE
Translated CVAP data saved to ./2015_cvap_diagg_2020/de/de_2020_cvap_block.csv
Processing state: FL
Translated CVAP data saved to ./2015_cvap_diagg_2020/fl/fl_2020_cvap_block.csv
Processing state: GA
Translated CVAP data saved to ./2015_cvap_diagg_2020/ga/ga_2020_cvap_block.csv


## Round the results to integers

In [ ]:
# Define the rounding function
def data_rounding(x):
    x = x.sort_values(ascending=False, kind='mergesort')
    floor = np.floor(x)
    diff = x - floor
    to_add = int(round(sum(x), 0) - sum(floor))
    floor[diff.nlargest(to_add).index] += 1
    return floor.sort_index(ascending=True).astype(int)

# Define paths
input_base_path = './2015_cvap_diagg_2020/'
output_base_path = './2015_cvap_disagg_2020_rounding/'

# Ensure the output directory exists
os.makedirs(output_base_path, exist_ok=True)

# Iterate over each state folder
for state in os.listdir(input_base_path):
    state_input_path = os.path.join(input_base_path, state)
    state_output_path = os.path.join(output_base_path, state)

    os.makedirs(state_output_path, exist_ok=True)

    for file in os.listdir(state_input_path):
        if file.endswith('.csv'):
            file_path = os.path.join(state_input_path, file)
            print(f"Processing file: {file_path}")

            data = pd.read_csv(file_path)

            # Round all columns besides GEOID
            columns_to_round = [col for col in data.columns if col not in ['GEOID20']]

            # Extract block groups
            data['BLKGRP'] = data['GEOID20'].astype(str).str[:12]

            # Perform rounding at the block group level
            rounded_data = []

            for col in columns_to_round:
                rounded_group = data.groupby('BLKGRP')[col].apply(data_rounding)

                # Flatten the group result into a DataFrame
                rounded_col = rounded_group.reset_index(level=0, drop=True)
                rounded_data.append(rounded_col)

            # Combine the rounded results with the original data
            for i, col in enumerate(columns_to_round):
                data[col] = rounded_data[i]

            # Drop temporary block group column
            data.drop(columns=['BLKGRP'], inplace=True)

            # Save the rounded data
            output_file_path = os.path.join(state_output_path, file)
            data.to_csv(output_file_path, index=False)
            print(f"Rounded data saved to {output_file_path}")


Processing file: ./2015_cvap_diagg_2020/vt/vt_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_rounding/vt/vt_2020_cvap_block.csv
Processing file: ./2015_cvap_diagg_2020/va/va_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_rounding/va/va_2020_cvap_block.csv
Processing file: ./2015_cvap_diagg_2020/sd/sd_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_rounding/sd/sd_2020_cvap_block.csv
Processing file: ./2015_cvap_diagg_2020/sc/sc_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_rounding/sc/sc_2020_cvap_block.csv
Processing file: ./2015_cvap_diagg_2020/ut/ut_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_rounding/ut/ut_2020_cvap_block.csv
Processing file: ./2015_cvap_diagg_2020/ga/ga_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_rounding/ga/ga_2020_cvap_block.csv
Processing file: ./2015_cvap_diagg_2020/ms/ms_2020_cvap_block.csv
Rounded data saved to ./2015_cvap_disagg_2020_roundi